<a href="https://colab.research.google.com/github/jettro/colab_notebooks/blob/main/Chatbot_telegram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyTelegramBotAPI
!pip install transformers
!pip install wikipedia

In [11]:
import wikipedia

def find_summary(description):
  try:
    return wikipedia.summary(description)
  except wikipedia.PageError:
    return "Nothing found!"

In [10]:
from transformers import pipeline

qa = pipeline(task = "question-answering")


No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


Downloading:   0%|          | 0.00/473 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/249M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [26]:
import telebot

bot_state = {
    "status": 0,
    "summary": ''
}


def listener(messages):
  for m in messages:
    chatid = m.chat.id
    if m.content_type == 'text':
      text = m.text
      if text == 'start':
        bot_state["status"] = 0
      
      if bot_state["status"] == 0:
        tb.send_message(chatid, 'hi there, what do you want to talk about?')
        bot_state["status"] = 1
      elif bot_state["status"] == 1:
        bot_state["summary"] = find_summary(text)
        if bot_state["summary"] == 'Nothing found!':
          tb.send_message(chatid, 'Unknown subject, please choose another')
        else:
          tb.send_message(chatid, 'What is your question?')
          bot_state["status"] = 2
      elif bot_state["status"] == 2:
        answer = qa(question = text, context = bot_state["summary"])
        print(answer)
        tb.send_message(chatid, answer["answer"])
        bot_state["status"] = 0



In [28]:
TOKEN = ''
chat_status = 0
summary = ''

tb = telebot.TeleBot(TOKEN)
tb.set_update_listener(listener)
tb.polling()


{'score': 0.09279080480337143, 'start': 168, 'end': 171, 'answer': 'ies'}


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:708: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py:301: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for span_id in range(num_spans)


{'score': 0.32165029644966125, 'start': 1708, 'end': 1724, 'answer': 'every four years'}
{'score': 0.6297376155853271, 'start': 28, 'end': 94, 'answer': 'an organized collection of data stored and accessed electronically'}
